# This notebook is used to scrape a Wikipedia page to obtain the data that is in a table of postal codes, and to transform it into a pandas DataFrame

We first import the relevant libraries needed for this exercise. To scrape the webpage, we will be using BeautifulSoup.

In [97]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [98]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

Lets take a look at the website in lxml format.

In [99]:
soup = BeautifulSoup(website_url,'lxml')
# print(soup.prettify())

We then scope into finding (soup.find()) the table we want, and notice it's a tagged as a wikitable sortable class.  

In [100]:
postalcode_table = soup.find('table',{'class':'wikitable sortable'})

Next, we get each table row (tr) to iterate through within the table.

In [101]:
rows = postalcode_table.find_all('tr')

In [102]:
pc_list = []
b_list = []
n_list = []
for row in rows:
    cells = row.find_all("td")
    try:
        postcode = cells[0].get_text()
        borough = cells[1].get_text()
        # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
        if 'Not assigned' in borough:
            continue
        # .strip() removes whitespace e.g. \n
        neighbourhood = cells[2].get_text().strip()
        # If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
        # So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
        if 'Not assigned' in neighbourhood:
            neighbourhood = borough
        pc_list.append(postcode)
        b_list.append(borough)
        n_list.append(neighbourhood)
    except:
        # do nothing
        continue

Finally, we convert our lists into a pandas DataFrame. We next merge identical postal code, and join the Neighbourhood for the same postal code with commas.

In [103]:
pc_df = pd.DataFrame()
pc_df['PostalCode'] = pc_list
pc_df['Borough'] = b_list
pc_df['Neighbourhood'] = n_list

# pc_df.groupby('PostalCode')['Neighbourhood'].apply(lambda x: "{%s}" % ', '.join(x))
result = pc_df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(', '.join)
# pc_df.head()
result_df = pd.DataFrame(result).reset_index()
# print(type(pc_df), type(result))
result_df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [104]:
print('Number of rows in the DataFrame: ', result_df.shape)

Number of rows in the DataFrame:  (103, 3)


The End.